In [43]:
def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
    """
    Learn value function and policy by using value iteration method for a given
    gamma and environment.

    Parameters:
    ----------
    p = {a1: [(p1, s'1, r1, terminal) ...]}
    P, nS, nA, gamma:
        defined at beginning of file
    tol: float
        Terminate value iteration when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns:
    ----------
    value_function: np.ndarray[nS]
    policy: np.ndarray[nS]
    """
    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)
    should_continue = True
    i = 0
    with open("value_iteration_history.txt", "w") as f:
        while should_continue:
            should_continue = False
            for s in range(nS):
                transitions = P[s]
                for action, action_ps in transitions.items():
                    v = 0
                    for action_p in action_ps:
                        p, s_prime, r, terminal = action_p
                        # NOTE: we are getting expectation of total rewards
                        v += p*(r + gamma * value_function[s_prime])
                    next_value = max(value_function[s], v)
                    # Pick the action that leads to the highest valued state.
                    if next_value == v:
                        policy[s] = action
                    if not should_continue and abs(next_value - value_function[s]) > tol:
                        should_continue = True
                    value_function[s] = next_value
            # f.write(f"======================{i}======================\n")
            # f.write("value function:\n")
            # f.write(str(value_function)+"\n")
            # f.write("policy:")
            # f.write(str(policy) + "\n")
            i += 1
        # for every s, find all its ps. and probabilities and next state
    return value_function, policy

def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
    value_function = -100*np.ones(nS)
    next_value_function = np.zeros(nS)
    i = 0
    while True:
        if all(np.abs(next_value_function - value_function) < tol):
            break
        value_function = next_value_function
        next_value_function = np.zeros(nS)
        for s in range(nS):
            action = policy[s]
            for transition in P[s][action]:
                p, next_state, r, done = transition
                next_value_function[s] += p*(r + gamma*value_function[next_state])
    return value_function

def policy_update(P, value_function, nS, nA, gamma=0.9):
    # Consider all expected r + gamma*V(s')
    # Q(s,a) = E[r(s, a, s') + gamma*V(s')]
    new_policy = np.zeros(nS, dtype=int)
    Q_sas = np.zeros(nS)
    for s in range(nS):
        for action, transitions in P[s].items():
            Q_sa = 0
            for transition in transitions:
                p, next_state, r, done = transition
                Q_sa += p * (r + gamma * value_function[next_state])
            if Q_sa > Q_sas[s]:
                new_policy[s] = action
                Q_sas[s] = Q_sa
    return new_policy
        

def policy_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
    policy = np.zeros(nS)
    i = 0
    while True:
        value_function = policy_evaluation(P, nS, nA, policy, gamma=gamma)
        new_policy = policy_update(P, value_function, nS, nA, gamma=gamma)
        #TODO Remember to remove
        print(f"======================{i}======================\n")
        print(f'Policy: {policy}')
        print(f'value function: {value_function}')
        if all(np.abs(new_policy - policy) < tol):
            break
        policy = new_policy
        i += 1
    return value_function, policy

    
def render_single(env, policy, state_output_file, max_steps=100):
    """
    env: gym.core.Environment - Environment to play on. Must have nS, nA, and P as attributes.
    Policy: np.array of shape [env.nS]. The action to take at a given state
    """ 
    episode_reward = 0
    ob = env.reset()
    try:
        os.remove(state_output_file)
    except FileNotFoundError:
        pass
    for t in range(max_steps):
        env.render(state_output_file)
        a = policy[ob]
        ob, rew, done, _ = env.step(a)
        episode_reward += rew
        if done:
            break
        time.sleep(0.25)
    env.render(state_output_file)
    if not done:
        print("The agent didn't reach a terminal state in {} steps.".format(max_steps))
    else:
        print("Episode reward: %f" % episode_reward)



Rico: current state 14, new state: 15
Rico: current state 55, new state: 63
Rico: current state 62, new state: 63

-------------------------
Beginning Policy Iteration
-------------------------
======================0======================

Policy: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
value function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
======================1======================

Policy: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
value function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
======================2======================

Policy: [0 0 0 0 0 0 0 0 0 0 1 0 0 2 2 0]
value function: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.9 1.  0. ]
======================3======================

Policy: [0 0 0 0 0 0 1 0 0 1 1 0 0 2 2 0]
value function: [0.   0.   0.   0.   0.   0.   0.81 0.   0.   0.81 0.9  0.   0.   0.9
 1.   0.  ]
======================4======================

Policy: [0 0 1 0 0 0 1 0 2 1 1 0 0 2 2 0]
value function: [0.    0.    0.72

## Value Iteration
Intro: Terminal State is 15, immediate state is 14. 
    - If there's no revisited states, and reward from goal state to goal state is 0,
    then value iteration will simply polulate from end pose back. Because r + gamma * V(s') = r at the second last state, and for rest of the states, r + gamma * V, with constant V for all other states.
    
WHY DOES IT ALWAYS FIND THE OPTIMAL POLICIES AND VALUES?
Let's call the bellman backup operator B, where $B^{\pi}V = R(s,a) + yV^{\pi}(s')$, following a policy $\pi$
- Does policy evaluation (see below) converge? Yes. One can prove $(BV1 - BV2 < |V1 - V2|_{\infty})$. policy evaluation always converges
- If you have two values functions that $V1(s) > V2(s)$, we can guarantee that $B^{\pi}V1 > B^{\pi}V2$
- If you take evaluate your value function, by "taking best action" at the current iteration, we call it bellman optimality operator, which is $B^{*}V = max_{action} R(s,a) + yV^{\pi}(s')$. We can prove: $B^{*}V > V$. That means, if you keep doing bellman optimality backup, your value function estimate will converge!
- Now the question: does it converge to optimal value? 
    - We know the optimal value function must exist, because we have **A FINITE SET OF POLICIES**.
    - Do we get the optimal value with bellman optimality backup? YES! Because $B^{*}V >= B^{\pi}V$ for any policy, $\pi$. So it's no worse than the best policy. and we know, the value function will converge.

```
======================0======================
value function:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
policy:[3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3]
======================1======================
value function:
[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.9 1.  0. ]
policy:[3 3 3 3 3 3 3 3 3 3 1 3 3 2 2 3]
======================2======================
value function:
[0.   0.   0.   0.   0.   0.   0.81 0.   0.   0.81 0.9  0.   0.   0.9
 1.   0.  ]
policy:[3 3 3 3 3 3 1 3 3 2 1 3 3 2 2 3]
======================3======================
value function:
[0.    0.    0.729 0.656 0.    0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]
policy:[3 3 1 0 3 3 1 3 2 2 1 3 3 2 2 3]
======================4======================
value function:
[0.    0.656 0.729 0.656 0.656 0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]
policy:[3 2 1 0 1 3 1 3 2 2 1 3 3 2 2 3]
======================5======================
value function:
[0.59  0.656 0.729 0.656 0.656 0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]
policy:[2 2 1 0 1 3 1 3 2 2 1 3 3 2 2 3]
======================6======================
value function:
[0.59  0.656 0.729 0.656 0.656 0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]
policy:[2 2 1 0 1 3 1 3 2 2 1 3 3 2 2 3]
```

## Policy Iteration Results

It's possible that initially, value function is zero everywhere. However, policy update can still break the tie because it considers expectation of total rewards, across all actions. 
Value iteration at the end will reach the final value function at optimal policy
```
======================0======================

Policy: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
value function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
======================1======================

Policy: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
value function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
======================2======================

Policy: [0 0 0 0 0 0 0 0 0 0 1 0 0 2 2 0]
value function: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 0.  0.  0.9 1.  0. ]
======================3======================

Policy: [0 0 0 0 0 0 1 0 0 1 1 0 0 2 2 0]
value function: [0.   0.   0.   0.   0.   0.   0.81 0.   0.   0.81 0.9  0.   0.   0.9
 1.   0.  ]
======================4======================

Policy: [0 0 1 0 0 0 1 0 2 1 1 0 0 2 2 0]
value function: [0.    0.    0.729 0.656 0.    0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]
======================5======================

Policy: [0 2 1 0 1 0 1 0 2 1 1 0 0 2 2 0]
value function: [0.    0.656 0.729 0.656 0.656 0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]
======================6======================

Policy: [1 2 1 0 1 0 1 0 2 1 1 0 0 2 2 0]
value function: [0.59  0.656 0.729 0.656 0.656 0.    0.81  0.    0.729 0.81  0.9   0.
 0.    0.9   1.    0.   ]

```